In [1]:
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_palette(sns.color_palette())

import numpy as np
import pandas as pd
import shogun as sg
from scipy.stats import invwishart, norm, multivariate_normal
from scipy.special import loggamma
from tqdm.notebook import tqdm
import multiprocessing
from itertools import repeat
import copy
import pickle

from mcmcjoint.samplers import *
from mcmcjoint.tests import *

In [2]:
seed = 12345
m_dim=1
num_trials = 20
nthreads = 10
num_samples = 1000
burn_in_samples = 250
sc_thinning_samples = 250
tau = 1
alpha = 0.05

In [3]:
for exper in range(1):
    if exper == 0:
        print(f'Experiment {exper}: no error')
        model = t_mixture_sampler
    else:
        print(f'Experiment {exper}: error')
        exec('model = t_mixture_sampler_error_'+str(exper))
    t_mix = model(D=m_dim, M=2, N=6, v=5,
        m_mu=[-2. * np.ones(m_dim), 2. * np.ones(m_dim)], 
        S_mu=[10. * np.identity(m_dim), 10. * np.identity(m_dim)], 
        v_Sigma=2.*np.array([1., 1.]), 
        Psi_Sigma=2.*np.array([1. * np.identity(m_dim), 1. * np.identity(m_dim)]), 
        alpha_p=np.array([1., 1.]))
    t_mix.set_nthreads(1)
    theta_indices = t_mix.theta_indices
    t_mix_exper = sample_experiment(num_trials=num_trials, nthreads=nthreads, seed=seed,
                                    sampler=t_mix, 
                                    num_samples=num_samples, 
                                    burn_in_samples=burn_in_samples,
                                    sc_thinning_samples=sc_thinning_samples, 
                                    tau=tau, alpha=alpha)
    out = t_mix_exper.run()
    results = [trial[0] for trial in out]
    samples = [trial[1] for trial in out]

    with open('./results/t_mixture/experiment'+'_' + str(exper) +'_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    with open('./samples/t_mixture/experiment_'+'_' + str(exper) +'_samples.pkl', 'wb') as f:
        pickle.dump(samples, f)
        